In [1]:
!pip list

Package                Version
---------------------- -------------------
absl-py                0.10.0
aiohttp                3.6.3
argon2-cffi            20.1.0
astor                  0.8.1
async-generator        1.10
async-timeout          3.0.1
attrs                  20.2.0
backcall               0.2.0
bayesian-optimization  1.2.0
bleach                 3.3.0
blinker                1.4
boto3                  1.17.46
botocore               1.20.78
brotlipy               0.7.0
cachetools             4.1.1
certifi                2020.12.5
cffi                   1.14.3
chardet                3.0.4
click                  7.1.2
cloudpickle            1.6.0
colorama               0.4.4
cryptography           3.1.1
cycler                 0.10.0
decorator              5.0.9
deepcut                0.7.0.0
defusedxml             0.7.1
entrypoints            0.3
et-xmlfile             1.1.0
Flask                  1.1.2
gast                   0.2.2
gensim                 3.6.0
google-auth      

In [1]:
!pip install pythainlp==2.3.1 --user

In [1]:
import deepcut
import pandas as pd
import numpy as np
import re
from itertools import chain
import scipy.sparse as sp
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from flask import Flask, jsonify, request
import threading 
import time
import json
import requests
import joblib
import pickle

## data from all category

In [3]:
def tokenize_text_list(ls):
    """Tokenize list of text"""
    return list(chain.from_iterable([deepcut.tokenize(ls)]))

In [4]:
exit_flag = True
beforeTok={}

def getQuestions():
    global beforeTok
    while True:
#         raw_questions = backendAPI()
        raw_questions = getQuestionsFromBackendAPI() # <-- Uncomment this to get real questions
        
        # do tokenize
        tokenize_questions = raw_questions
        
        curriculumDF = pd.DataFrame(data=tokenize_questions['หลักสูตร'])
        curriculumDF = curriculumDF.rename(columns={0:"curriculum"})
        
        admissionDF = pd.DataFrame(data=tokenize_questions['การรับเข้านักศึกษา'])
        admissionDF = admissionDF.rename(columns={0:"admission"})
        
        enrollmentDF = pd.DataFrame(data=tokenize_questions['ลงทะเบียนเรียน'])
        enrollmentDF = enrollmentDF.rename(columns={0:"enrollment"})
        
        faqDF = pd.DataFrame(data=tokenize_questions['คำถามทั่วไป'])
        faqDF = faqDF.rename(columns={0:"faq"})
        
        tokenized_enrollment =enrollmentDF.enrollment.map(tokenize_text_list)
        tokenized_admission =admissionDF.admission.map(tokenize_text_list)
        tokenized_curriculum =curriculumDF.curriculum.map(tokenize_text_list)
        tokenized_faq =faqDF.faq.map(tokenize_text_list)
        
#         beforeTok = {}
        beforeTok['ลงทะเบียนเรียน'] = enrollmentDF
        beforeTok['การรับเข้านักศึกษา'] = admissionDF
        beforeTok['หลักสูตร'] = curriculumDF
        beforeTok['คำถามทั่วไป'] = faqDF
        
        # Update question
        questions_data = tokenize_questions
        if exit_flag: 
            break
        
        # Set query time
        time.sleep(259200)

In [5]:
url = 'https://natthawat.live/api'

In [6]:
def getQuestionsFromBackendAPI():
    response = requests.get('%s/km/faq' % url)
    faqs = json.loads(response.text)

    response = requests.get('%s/km/category' % url)
    categories = json.loads(response.text)

    questions_data = {}
    for category in categories:
        questions_data[category['category']] = []
        for faq in faqs:
            if faq['category']['category'] == category['category']:
                questions_data[category['category']].append(faq['question'])
                
    return questions_data

In [7]:
getQuestionsThread = threading.Thread(target = getQuestions)

In [8]:
getQuestionsThread.start()

In [9]:
%%time
getQuestions()

Wall time: 30 s


## category

In [10]:
data = pd.read_excel("Category.xlsx")

In [11]:
#Load File
with open('token_text_category.data', 'rb') as filehandle:
    # read the data as binary data stream
    tokenized_texts = pickle.load(filehandle)

In [12]:
def text_to_bow(tokenized_text, vocabulary_):
    n_doc = len(tokenized_text)
    values, row_indices, col_indices = [], [], []
    for r, tokens in enumerate(tokenized_text):
        feature = {}
        for token in tokens:
            word_index = vocabulary_.get(token)
            if word_index is not None:
                if word_index not in feature.keys():
                    feature[word_index] = 1
                else:
                    feature[word_index] += 1
        for c, v in feature.items():
            values.append(v)
            row_indices.append(r)
            col_indices.append(c)
        #print(feature)

    # document-term matrix in sparse CSR format
    X = sp.csr_matrix((values, (row_indices, col_indices)),
                      shape=(n_doc, len(vocabulary_)))
    return X

vocabulary_ = {v: k for k, v in enumerate(set(chain.from_iterable(tokenized_texts)))}
X = text_to_bow(tokenized_texts, vocabulary_)

In [13]:
transformer = TfidfTransformer()
svd_model = TruncatedSVD(n_components=100,
                         algorithm='arpack', n_iter=100)
X_tfidf = transformer.fit_transform(X)
X_svd = svd_model.fit_transform(X_tfidf)

In [14]:
tag = pd.get_dummies(data.Category).columns

In [15]:
#Load Model
logist_models = joblib.load("category_model.pkl")

C:\Users\NATTHAWATTUNGRUETHAI\.conda\envs\tensorflow\lib\site-packages\sklearn\base.py:315: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [16]:
y_pred = np.argmax(np.vstack([model.predict_proba(X_svd)[:, 1] for model in logist_models]).T, axis=1)
y_pred = np.array([tag[yi] for yi in y_pred])
y_true = data.Category.values
print(tag[0:4])

Index(['การรับเข้านักศึกษา', 'คำถามทั่วไป', 'ลงทะเบียนเรียน', 'หลักสูตร'], dtype='object')


In [17]:
#Clean Text
def remove_repettition(text):
    token_list = list(text)
    if len(token_list) > 2:
        filter_list = [True, True]
        n = len(token_list)
        for i in range(2, n):
            if (token_list[i] == token_list[i-1]) and (token_list[i] == token_list[i-2]):
                filter_list.append(False)
            else:
                filter_list.append(True)

        output = ''.join(np.array(token_list)[filter_list])
    else:
        output = text
    return output

def cleansing(text):
    # \t, \n, \xa0 and other special characters. Replace by blank string
    text = re.sub('[\t\n\xa0\"\'!?\/\(\)%\:\=\-\+\*\_ๆ]', '', text)
    
    # Numbers. Replace by space
    text = re.sub('[0-9]', ' ', text)
    
    # Dot. Replace by space
    text = re.sub('[\.]', ' ', text)
    
    # One or more consecutive space. Replace by single space
    text = re.sub('\s+',' ',text)
    
    # Remove 2 or more repettition
    text = remove_repettition(text)
    
    return text

In [18]:
textInput = 'วิศวคอมพิวเตอร์มีหลักสูตรอะไรบ้าง'
textInput= cleansing(textInput)
tokenized_text = deepcut.tokenize(textInput)
x = text_to_bow([tokenized_text], vocabulary_)
x_tfidf = transformer.transform(x)
x_svd = svd_model.transform(x_tfidf)
pred = [model.predict_proba(x_svd.reshape(-1, 1).T).ravel()[1] for model in logist_models]
print(list(zip(tag, pred)))
predict_category = max(list(zip(tag, pred)))

[('การรับเข้านักศึกษา', 0.15501360454034974), ('คำถามทั่วไป', 0.23211256402801916), ('ลงทะเบียนเรียน', 0.08012019772465437), ('หลักสูตร', 0.4839496158840339)]


In [19]:
max_value = 0
max_category = ''
pred_results = list(zip(tag, pred))
for pred_result in pred_results:
  # print(pred_result)
  if pred_result[1] > max_value:
    max_value = pred_result[1]
    max_category = pred_result[0]
print(max_category, max_value)

หลักสูตร 0.4839496158840339


## prediction

In [20]:
#cosine
from pythainlp import word_tokenize 
from pythainlp.word_vector import * 
from sklearn.metrics.pairwise import cosine_similarity  
model=get_model()
def sentence_vectorizer(ss,dim=300,use_mean=True): 
    s = word_tokenize(ss)
    vec = np.zeros((1,dim))
    for word in s:
        if word in model.wv.index2word:
            vec+= model.wv.word_vec(word)
        else: pass
    if use_mean: vec /= len(s)
    return vec
def sentence_similarity(s1,s2):
    return cosine_similarity(sentence_vectorizer(str(s1)),sentence_vectorizer(str(s2)))

C:\Users\NATTHAWATTUNGRUETHAI\AppData\Roaming\Python\Python37\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [21]:
len(beforeTok[max_category])

82

In [22]:
word = beforeTok[max_category].iloc[0]

In [23]:
word[0]

'วิชาภาคบังคับที่มีวิชาต่อเนื่อง มีอะไรบ้าง หลักสูตรปกติ?'

In [24]:
word

curriculum    วิชาภาคบังคับที่มีวิชาต่อเนื่อง มีอะไรบ้าง หลั...
Name: 0, dtype: object

In [25]:
import json

In [26]:
result = []
for index in range(len(beforeTok[max_category])):
#     print(beforeTok[max_category].iloc[index])
    word = beforeTok[max_category].iloc[index]
#     print(word[0])
#     word = beforeTok[max_category].iloc[index]
    subResult = sentence_similarity(textInput,word[0])
    subResult = subResult[0][0]
    result.append(subResult)

<ipython-input-20-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-20-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.word_vec(word)
<ipython-input-20-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-20-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.word_vec(word)
<ipython-input-20-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-20-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.w

<ipython-input-20-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-20-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.word_vec(word)
<ipython-input-20-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-20-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.word_vec(word)
<ipython-input-20-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-20-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.w

In [27]:
max_question_percentage = max(result)
print(max_question_percentage)
question_index = np.where(result == max_question_percentage)
print(question_index)
predictedQuestion = beforeTok[max_category].loc[question_index[0][0]]
predictedQuestion = predictedQuestion[0]

0.7615871650925202
(array([74], dtype=int64),)


In [28]:
predictedQuestion

'วิชาภาคบังคับที่มีตัวต่อเนื่องมีอะไรบ้าง หลักสูตรนานาชาติ?'

# time total

In [29]:
%%time
textInput = "วิศวคอมมีหลักสูตรอะไรบ้าง"
textInput= cleansing(textInput)
tokenized_text = deepcut.tokenize(textInput)
x = text_to_bow([tokenized_text], vocabulary_)
x_tfidf = transformer.transform(x)
x_svd = svd_model.transform(x_tfidf)
pred = [model.predict_proba(x_svd.reshape(-1, 1).T).ravel()[1] for model in logist_models]
print(list(zip(tag, pred)))
predict_category = max(list(zip(tag, pred)))

max_value = 0
max_category = ''
pred_results = list(zip(tag, pred))
for pred_result in pred_results:
  # print(pred_result)
    if pred_result[1] > max_value:
        max_value = pred_result[1]
        max_category = pred_result[0]
print(max_category, max_value)

result = []
for index in range(len(beforeTok[max_category])):
    word = beforeTok[max_category].iloc[index]
    subResult = sentence_similarity(textInput,word[0])
    subResult = subResult[0][0]
    result.append(subResult)
max_question_percentage = max(result)
print(max_question_percentage)
question_index = np.where(result == max_question_percentage)
print(question_index)
#         print(beforeTok[max_category].loc[question_index[0][0]])
predictedQuestion = beforeTok[max_category].loc[question_index[0][0]]
predictedQuestion = predictedQuestion[0]
print(predictedQuestion)
value = {
    "category": max_category,
    "accuracy": "%lf" % max_value,
    "predictedQuestion": str(predictedQuestion),
    "similarity": "%lf" % max_question_percentage
}

[('การรับเข้านักศึกษา', 0.1542952454216094), ('คำถามทั่วไป', 0.2892175898928962), ('ลงทะเบียนเรียน', 0.07409858686178904), ('หลักสูตร', 0.43034208776352484)]
หลักสูตร 0.43034208776352484


<ipython-input-20-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-20-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.word_vec(word)
<ipython-input-20-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-20-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.word_vec(word)
<ipython-input-20-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-20-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.w

<ipython-input-20-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-20-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.word_vec(word)
<ipython-input-20-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-20-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.word_vec(word)
<ipython-input-20-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-20-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.w

0.7717235367264967
(array([37], dtype=int64),)
วิศวะคอม/ มีทั้งหมดกี่หลักสูตร? มีหลักสูตรอะไรบ้าง?
Wall time: 807 ms


# time seperate

In [30]:
%%time
textInput = "วิศวคอมมีหลักสูตรอะไรบ้าง"
textInput= cleansing(textInput)

Wall time: 0 ns


In [31]:
%%time
tokenized_text = deepcut.tokenize(textInput)

Wall time: 59.8 ms


In [32]:
%%time
x = text_to_bow([tokenized_text], vocabulary_)
x_tfidf = transformer.transform(x)
x_svd = svd_model.transform(x_tfidf)

Wall time: 2 ms


In [33]:
%%time
pred = [model.predict_proba(x_svd.reshape(-1, 1).T).ravel()[1] for model in logist_models]
# print(list(zip(tag, pred)))

Wall time: 1.99 ms


In [34]:
%%time
predict_category = max(list(zip(tag, pred)))
max_value = 0
max_category = ''
pred_results = list(zip(tag, pred))

Wall time: 0 ns


In [35]:
%%time
for pred_result in pred_results:
  # print(pred_result)
    if pred_result[1] > max_value:
        max_value = pred_result[1]
        max_category = pred_result[0]
print(max_category, max_value)
# end of category model

หลักสูตร 0.43034208776352484
Wall time: 998 µs


In [36]:
%%time
result = []
for index in range(len(beforeTok[max_category])):
    word = beforeTok[max_category].iloc[index]
    subResult = sentence_similarity(textInput,word[0])
    subResult = subResult[0][0]
    result.append(subResult)

<ipython-input-20-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-20-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.word_vec(word)
<ipython-input-20-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-20-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.word_vec(word)
<ipython-input-20-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-20-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.w

<ipython-input-20-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.word_vec(word)
<ipython-input-20-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-20-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.word_vec(word)
<ipython-input-20-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-20-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.word_vec(word)
<ipython-input-20-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.w

Wall time: 761 ms


<ipython-input-20-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.word_vec(word)
<ipython-input-20-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-20-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.word_vec(word)
<ipython-input-20-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.index2word:
<ipython-input-20-b88f1432073f>:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vec+= model.wv.word_vec(word)
<ipython-input-20-b88f1432073f>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.w

In [37]:
%%time
max_question_percentage = max(result)
# print(max_question_percentage)
question_index = np.where(result == max_question_percentage)
# print(question_index)
# print(beforeTok[max_category].loc[question_index[0][0]])
predictedQuestion = beforeTok[max_category].loc[question_index[0][0]]
predictedQuestion = predictedQuestion[0]
# print(predictedQuestion)

Wall time: 0 ns


In [38]:
%%time
value = {
    "category": max_category,
    "accuracy": "%lf" % max_value,
    "predictedQuestion": str(predictedQuestion),
    "similarity": "%lf" % max_question_percentage
}

Wall time: 0 ns
